# Independent Q-learning

> In this notebook, we implement the **[Independent Q learning]()** Multi Agent Reinforcement Leaning (MARL) algorithm in our environment. 


> Tutorial based on [IQL TorchRL Tutorial](https://github.com/pytorch/rl/blob/main/sota-implementations/multiagent/iql.py).

#### High-level overview of IQL algorithm

In IQL a centralized state-action value function is used, Q<sub>tot</sub>, and each agent α learns an individual action-value function Q<sub>α</sub>, independently.

### Simulation overview

> We simulate our environment with an initial population of **20 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **10 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ the Independent Q learning reinforcement learning algorithm to further refine their learning.

#### Imported libraries

In [1]:
import sys
import os
import os
import time
import torch

from tensordict.nn import TensorDictModule, TensorDictSequential
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torch import nn
from torchrl._utils import logger as torchrl_logger
from torchrl.collectors import SyncDataCollector
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import EGreedyModule, QValueModule, SafeSequential
from torchrl.modules.models.multiagent import MultiAgentMLP
from torchrl.objectives import SoftUpdate, ValueEstimators, DQNLoss

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from RouteRL.environment.environment import TrafficEnvironment
from RouteRL.utilities import get_params

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#### Hyperparameters setting

In [2]:
# Devices
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

print("device is: ", device)
vmas_device = device  # The device where the simulator is run

# Sampling
frames_per_batch = 40  # Number of team frames collected per training iteration
n_iters = 20  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 100  # Number of optimization steps per training iteration
minibatch_size = 2  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients
memory_size = 1000  # Size of the replay buffer
tau =  0.005

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.99  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss

device is:  cpu


#### Environment initialization

> In this example, the environment initially contains only human agents.

In [3]:
params = get_params("params.json")
env = TrafficEnvironment(params, generate_agent_data=True, generate_paths=True)

print(env)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: /opt/homebrew/opt/sumo/share/sumo/tools
TrafficEnvironment with 20 agents.            
0 machines and 20 humans.            
Machines: []            
Humans: [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19]


> Reset the environment and the connection with SUMO

In [4]:
env.start()
env.reset()

 Retrying in 1 seconds


({}, {})

#### Human learning

In [5]:
num_episodes = 100

for episode in range(num_episodes):
    env.step()

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles). You can adjust the number of agents to be mutated in the <code style="color:white">/params.json</code> file.

In [6]:
env.mutation()
print(env)

TrafficEnvironment with 20 agents.            
10 machines and 10 humans.            
Machines: [Machine 11, Machine 18, Machine 13, Machine 3, Machine 6, Machine 1, Machine 4, Machine 0, Machine 2, Machine 9]            
Humans: [Human 5, Human 7, Human 8, Human 10, Human 12, Human 14, Human 15, Human 16, Human 17, Human 19]


> Create a group that contains all the machine (RL) agents.

>  **Hint:** the agents aren't competely independent in this example.

#### PettingZoo environment wrapper

In [7]:
group = {'agents': [str(machine.id) for machine in env.machine_agents]}

env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

> The environment is defined by a series of metadata that describe what can be expected during its execution. 

There are four specs to look at:

- <code style="color:white">action_spec</code> defines the action space;

- <code style="color:white">reward_spec</code> defines the reward domain;

- <code style="color:white">done_spec</code> defines the done domain;

- <code style="color:white">observation_spec</code> which defines the domain of all other outputs from environment steps;

In [8]:
print("action_spec:", env.full_action_spec, "\n\n")
print("reward_spec:", env.full_reward_spec, "\n\n")
print("done_spec:", env.full_done_spec, "\n\n")
print("observation_spec:", env.observation_spec, "\n\n")

action_spec: Composite(
    agents: Composite(
        action: Categorical(
            shape=torch.Size([10]),
            space=CategoricalBox(n=3),
            device=cpu,
            dtype=torch.int64,
            domain=discrete),
        device=cpu,
        shape=torch.Size([10])),
    device=cpu,
    shape=torch.Size([])) 


reward_spec: Composite(
    agents: Composite(
        reward: UnboundedContinuous(
            shape=torch.Size([10, 1]),
            space=ContinuousBox(
                low=Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, contiguous=True),
                high=Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, contiguous=True)),
            device=cpu,
            dtype=torch.float32,
            domain=continuous),
        device=cpu,
        shape=torch.Size([10])),
    device=cpu,
    shape=torch.Size([])) 


done_spec: Composite(
    done: Categorical(
        shape=torch.Size([1]),
        space=CategoricalBox(n=2),


#### Transforms

> We can append any TorchRL transform we need to our environment. These will modify its input/output in some desired way. In multi-agent contexts, it is paramount to provide explicitly the keys to modify.



Here we instatiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [9]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [10]:
check_env_specs(env)
env.reset()

2025-01-28 14:56:37,117 [torchrl][INFO] check_env_specs succeeded!


TensorDict(
    fields={
        agents: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([10]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_

#### Policy network

> Instantiate an `MPL` that can be used in multi-agent contexts.

In [11]:
net = MultiAgentMLP(
        n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs=env.action_spec.space.n,
        n_agents=env.n_agents,
        centralised=False,
        share_params=True,
        device=device,
        depth=2,
        num_cells=256,
        activation_class=nn.Tanh,
    )

> The neural network is wrapped in a `TensorDictModule`, which is responsible for managing the input and output interactions with the tensordict. Specifically, the module reads from the specified `in_keys`, processes the inputs through the neural network, and writes the resulting outputs to the defined `out_keys`. 

In [12]:
module = TensorDictModule(
        net, in_keys=[("agents", "observation")], out_keys=[("agents", "action_value")]
)

> **`QValueModule`** takes a tensor as input, which contains the `Q-values` (these values indicate how good it is to take each action in the given state). It identifies the action with the highest `Q-values` using the `argmax` operation.

In [13]:
value_module = QValueModule(
    action_value_key=("agents", "action_value"),
    out_keys=[
        env.action_key,
        ("agents", "action_value"),
        ("agents", "chosen_action_value"),
    ],
    spec=env.action_spec,
    action_space=None,
)

> **`SafeSequential`** is a `TensordictModule` that will concatenate the parameter lists in a single list.

In [14]:
qnet = SafeSequential(module, value_module)

> In the already made `Q network` the **`Epsilon-Greedy exploration module`** is added. This module randomly updates the actions in a tensordict given an epsilon greedy exploration strategy.

In [15]:
qnet_explore = TensorDictSequential(
    qnet,
    EGreedyModule(
        eps_init=0.3,
        eps_end=0,
        annealing_num_steps=int(total_frames * (1 / 2)),
        action_key=env.action_key,
        spec=env.action_spec,
    ),
)

#### Collector

Collectors perform the following operations:

1. **Reset Environment**: Initialize the environment.
2. **Compute Action**: Determine the next action using the policy and the latest observation.
3. **Execute Step**: Step through the environment with the computed action.

These operations repeat until the environment signals to stop.

In [16]:
collector = SyncDataCollector(
        env,
        qnet_explore,
        device=device,
        storing_device=device,
        frames_per_batch=frames_per_batch,
        total_frames=total_frames,
    )

#### Replay buffer

> In an off-policy setting, the replay buffer exceeds the number of frames utilized for policy updates, allowing agents to learn from previous rollouts as well.



In [17]:
replay_buffer = TensorDictReplayBuffer(
        storage=LazyTensorStorage(memory_size, device=device),
        sampler=SamplerWithoutReplacement(),
        batch_size=minibatch_size,
    )

#### DQN loss function

In [18]:
loss_module = DQNLoss(qnet, delay_value=True)

loss_module.set_keys(
        action_value=("agents", "action_value"),
        action=env.action_key,
        value=("agents", "chosen_action_value"),
        reward=env.reward_key,
        done=("agents", "done"),
        terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(ValueEstimators.TD0, gamma=gamma)
target_net_updater = SoftUpdate(loss_module, eps=1 - tau)

optim = torch.optim.Adam(loss_module.parameters(), lr)


#### Training loop

In [19]:
total_time = 0
total_frames = 0
sampling_start = time.time()


for i, tensordict_data in enumerate(collector):
    torchrl_logger.info(f"\nIteration {i}")

    sampling_time = time.time() - sampling_start

    current_frames = tensordict_data.numel()
    total_frames += current_frames
    data_view = tensordict_data.reshape(-1)
    replay_buffer.extend(data_view)
    

    training_tds = []
    training_start = time.time()

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)
            training_tds.append(loss_vals.detach())

            loss_value = loss_vals["loss"]

            loss_value.backward()

            total_norm = torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), max_grad_norm
            )
            training_tds[-1].set("grad_norm", total_norm.mean())

            optim.step()
            optim.zero_grad()
            target_net_updater.step()

    qnet_explore[1].step(frames=current_frames)  # Update exploration annealing
    collector.update_policy_weights_()

    training_time = time.time() - training_start

    iteration_time = sampling_time + training_time
    total_time += iteration_time
    training_tds = torch.stack(training_tds) 

collector.shutdown()

2025-01-28 14:56:39,654 [torchrl][INFO] 
Iteration 0
2025-01-28 14:56:43,920 [torchrl][INFO] 
Iteration 1
2025-01-28 14:56:48,081 [torchrl][INFO] 
Iteration 2
2025-01-28 14:56:52,225 [torchrl][INFO] 
Iteration 3
2025-01-28 14:56:56,432 [torchrl][INFO] 
Iteration 4
2025-01-28 14:57:00,538 [torchrl][INFO] 
Iteration 5
2025-01-28 14:57:04,718 [torchrl][INFO] 
Iteration 6
2025-01-28 14:57:08,849 [torchrl][INFO] 
Iteration 7
2025-01-28 14:57:13,048 [torchrl][INFO] 
Iteration 8
2025-01-28 14:57:17,190 [torchrl][INFO] 
Iteration 9
2025-01-28 14:57:21,339 [torchrl][INFO] 
Iteration 10
2025-01-28 14:57:25,483 [torchrl][INFO] 
Iteration 11
2025-01-28 14:57:29,668 [torchrl][INFO] 
Iteration 12
2025-01-28 14:57:33,811 [torchrl][INFO] 
Iteration 13
2025-01-28 14:57:38,007 [torchrl][INFO] 
Iteration 14
2025-01-28 14:57:42,306 [torchrl][INFO] 
Iteration 15
2025-01-28 14:57:46,567 [torchrl][INFO] 
Iteration 16
2025-01-28 14:57:50,858 [torchrl][INFO] 
Iteration 17
2025-01-28 14:57:55,041 [torchrl][INFO

In [20]:
env.plot_results()